In [1]:
import pandas as pd
import numpy as np
departments = pd.read_csv('departments.csv', sep=',', encoding="ISO-8859-1")
products = pd.read_csv('products.csv', sep=',', encoding="ISO-8859-1")
mergedOrders = pd.read_csv('mergedOrders.csv', sep=',', encoding="ISO-8859-1")
orders1 = pd.read_csv('orders.csv', sep=',', encoding="ISO-8859-1")

In [2]:
products.head()
departments.head()
mergedOrders.head()

,Unnamed: 0,order_id,product_id,reordered,user_id,days_since_prior_order
0,0,8,23423,1,3107,17.0
1,1,40,10070,1,382,23.0
2,2,40,42450,1,382,23.0
3,3,40,33198,1,382,23.0
4,4,40,34866,1,382,23.0


In [3]:
#mergedOrders = mergedOrders.drop(columns = ["order_id"])
dfDaysAverage = mergedOrders.groupby("product_id").mean()
dfDaysAverage.sort_values("days_since_prior_order").dropna()
dfDaysAverage = dfDaysAverage.drop(columns=["Unnamed: 0","order_id","reordered","user_id"])
dfDaysAverage = dfDaysAverage.rename(columns={"days_since_prior_order": "mediaEcompras"})
dfDaysAverage = dfDaysAverage[dfDaysAverage["mediaEcompras"] > 0]
dfDaysAverage['product_id'] = dfDaysAverage.index
dfDaysAverage.head()

,mediaEcompras,product_id
product_id,,
1,12.333333,1
2,20.500000,2
3,19.250000,3
4,12.000000,4
7,10.666667,7


In [4]:
dfQuantity = mergedOrders.groupby("product_id").count()
dfQuantity = dfQuantity.drop(columns=["Unnamed: 0","order_id","reordered","days_since_prior_order"]).reset_index()
dfQuantity = dfQuantity.rename(columns={"user_id": "cantitatMitjaProductes"})
dfQuantity = dfQuantity.sort_values('cantitatMitjaProductes')
dfQuantity['product_id'] = dfQuantity.index
dfQuantity.head()

,product_id,cantitatMitjaProductes
20144,20144,1
28037,28037,1
13241,13241,1
13242,13242,1
13243,13243,1


In [5]:
allMergedOrders = mergedOrders
aux = allMergedOrders.groupby( [ "user_id", "product_id"] ).size().to_frame(name = 'total').reset_index()
allMergedOrders = allMergedOrders.groupby( [ "product_id", "user_id"] ).mean().reset_index()
allMergedOrders = allMergedOrders.drop(columns=["Unnamed: 0","order_id","reordered"]).reset_index()
allMergedOrders = pd.merge(aux,allMergedOrders, on=[ "user_id", "product_id"])
allMergedOrders = allMergedOrders.rename(columns={"days_since_prior_order": "media_usuarioEcompras"})
allMergedOrders = allMergedOrders.drop(columns=["index"])
allMergedOrders.head()

,user_id,product_id,total,media_usuarioEcompras
0,1,196,11,19.000000
1,1,10258,10,19.000000
2,1,10326,1,28.000000
3,1,12427,10,19.555556
4,1,13032,4,19.750000


In [39]:
def puntTemps(df):
    a2 = df['mediaEcompras'] / 2

    if df['media_usuarioEcompras'] <= df['mediaEcompras']: 
        return 5
    elif df['media_usuarioEcompras'] < df['mediaEcompras'] + a2:
        return 3
    else: 
        return 1

def puntQuant(df):
    a2 = df['cantitatMitjaProductes'] / 2
    
    if df['total'] < a2: 
        return 1
    elif df['total'] < df['cantitatMitjaProductes']:
        return 3
    else: 
        return 5

In [40]:
dfDaysAverage.index.name = None
dfProva = allMergedOrders
#dfProva = dfProva.groupby(["user_id",'product_id']).agg({'product_id':['mean', 'median']})
#dfProva['puntuacio'] = dfProva.age * dfProva.bmi
#dfProva = dfProva.groupby( [ "user_id", "product_id"] ).size().to_frame(name = 'total').reset_index()
dfProva =  pd.merge(dfProva, dfQuantity, on="product_id")
dfProva =  pd.merge(dfProva, dfDaysAverage, on="product_id")
dfProva['punCant'] = dfProva.apply(puntQuant,axis=1)
dfProva['punTemps'] = dfProva.apply(puntTemps,axis=1)
dfProva['punTotal'] = dfProva.punTemps + dfProva.punCant
dfProva

,user_id,product_id,total,media_usuarioEcompras,cantitatMitjaProductes,mediaEcompras,punCant,punTemps,punTotal
0,1,196,11,19.000000,9,12.160839,5,1,6
1,15,196,5,8.600000,9,12.160839,3,5,8
2,19,196,3,6.666667,9,12.160839,1,5,6
3,21,196,1,6.000000,9,12.160839,1,5,6
4,31,196,2,7.500000,9,12.160839,1,5,6
...,...,...,...,...,...,...,...,...,...
245292,6038,4078,1,5.000000,39,5.000000,1,5,6
245293,6038,19526,1,15.000000,2,15.000000,3,5,8
245294,6038,20096,2,7.500000,1,7.500000,5,5,10
245295,6043,21628,1,30.000000,4,30.000000,1,5,6


In [41]:
dfTotal = dfQuantity
dfTotal = pd.merge(dfDaysAverage, dfTotal, on = "product_id")

In [42]:
dfTotal = dfTotal.sort_values('cantitatMitjaProductes')
dfTotal

,mediaEcompras,product_id,cantitatMitjaProductes
12369,9.125000,20334,1
11605,13.500000,19025,1
11611,26.000000,19032,1
11612,11.600000,19033,1
4279,20.666667,7046,1
...,...,...,...
18280,15.000000,29910,5065
18054,14.335227,29553,6813
8307,30.000000,13695,7115
7992,23.000000,13191,8182


In [43]:
dfPuntUsuariTotal = dfProva.drop(columns=["total","media_usuarioEcompras","cantitatMitjaProductes", "mediaEcompras"])
dfPuntUsuariTotal

,user_id,product_id,punCant,punTemps,punTotal
0,1,196,5,1,6
1,15,196,3,5,8
2,19,196,1,5,6
3,21,196,1,5,6
4,31,196,1,5,6
...,...,...,...,...,...
245292,6038,4078,1,5,6
245293,6038,19526,3,5,8
245294,6038,20096,5,5,10
245295,6043,21628,1,5,6


In [78]:
from sklearn.model_selection import train_test_split

ratings = dfPuntUsuariTotal
train, test = train_test_split(ratings, test_size = 0.10)
ratings = pd.merge(dfPuntUsuariTotal, test)


ratings.groupby('product_id').count().sort_values('user_id')

,user_id,punCant,punTemps,punTotal
product_id,,,,
15351,1,1,1,1
17256,1,1,1,1
17247,1,1,1,1
17245,1,1,1,1
17242,1,1,1,1
...,...,...,...,...
26209,130,130,130,130
21137,153,153,153,153
21903,169,169,169,169


In [45]:
userRatings = ratings.pivot_table(index=['user_id'], columns=['product_id'], values='punTotal')

In [79]:
userRatings[24852].dropna()

user_id
28       8.0
116      6.0
136     10.0
182     10.0
188     10.0
        ... 
5892    10.0
5937    10.0
6016    10.0
6038     6.0
6042     6.0
Name: 24852, Length: 207, dtype: float64

In [48]:
corrMatrix = userRatings.corr(method='pearson')
corrMatrix

product_id,1,4,8,10,12,23,25,28,29,31,...,31042,31049,31060,31066,31073,31075,31078,31081,31087,31091
product_id,,,,,,,,,,,,,,,,,,,,,
1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
31078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31081,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
corrMatrix.to_csv("Matrix.txt")

In [98]:
#myRatings = userRatings.loc[0].dropna()
#myRatings
myRatings = {24852: 10}
myRatings
corrMatrix[24852].dropna()

product_id
34       1.000000
432      0.258199
1158     1.000000
1940     1.000000
2295     0.866025
           ...   
28849   -1.000000
28928    1.000000
29487    0.907959
30021   -1.000000
30720    1.000000
Name: 24852, Length: 87, dtype: float64

In [99]:
simCandidates = pd.Series()
print(simCandidates)
for key in myRatings:
    print ("Añadiendo productos similares a " + str(key) + "...")
    sims = corrMatrix[key].dropna()
    sims = sims.map(lambda x: x * myRatings[key])
    simCandidates = simCandidates.append(sims)
    

print ("ordenando...")
simCandidates.sort_values(inplace = True, ascending = False)
print (simCandidates.head(1000))

Series([], dtype: float64)
Añadiendo productos similares a 24852...
ordenando...
30720    10.0
10814    10.0
20580    10.0
19706    10.0
19678    10.0
         ... 
12569   -10.0
15613   -10.0
30021   -10.0
17794   -10.0
11136   -10.0
Length: 87, dtype: float64


<ipython-input-99-a526dbb9b3f6>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  simCandidates = pd.Series()


In [100]:
simCandidates = simCandidates.groupby(simCandidates.index).sum()

In [101]:
simCandidates.sort_values(inplace = True, ascending = False)
simCandidates.head(10)
#items = simCandidates.items()
#for item in items:
#   print(item[0])

30720    10.0
10814    10.0
20580    10.0
19706    10.0
19678    10.0
19003    10.0
18441    10.0
17828    10.0
16349    10.0
15683    10.0
dtype: float64

In [102]:
print(myRatings.keys())

dict_keys([24852])


In [95]:
filteredSims = simCandidates.drop(myRatings.keys())
filteredSims.head(10)
#print(simCandidates.to_string())

30720    1.0
10814    1.0
20580    1.0
19706    1.0
19678    1.0
19003    1.0
18441    1.0
17828    1.0
16349    1.0
15683    1.0
dtype: float64